# 🧠 Script 07 — Dense Retrieval + Query Understanding

## Pourquoi cette étape dans notre progression ?

Après avoir établi une baseline **BM25** (requête brute), puis testé :
- l'**enrichissement de requête** (query understanding) côté BM25,
- et le **filtrage métier** du corpus,

on passe ici à l’étape suivante : **remplacer le retriever lexical par un retriever sémantique**, tout en **conservant l’enrichissement métier**.

L’idée est de tester l’hypothèse suivante :

> Quand la requête utilisateur est ambiguë ou “non juridique” dans sa formulation,  
> l’enrichissement métier fournit des concepts stables, et le dense retrieval peut alors exploiter la similarité sémantique pour mieux retrouver les bons passages.

✅ Protocole strict (comparabilité) :
- même corpus `documents` (filtré “Code du travail”),
- mêmes questions (`benchmark_queries`),
- mêmes métriques (Recall@10, MRR, nDCG@10),
- **seul le retriever change** (BM25 → dense embeddings) et on conserve la couche “métier”.

---
## 🧠 Embedder utilisé (Dense Retrieval)

Dans ce stage, le retrieval dense repose sur l’embedder :

- **Embedder (Sentence-Transformers)** : `all-MiniLM-L6-v2` *(384 dimensions, rapide, baseline robuste)*

### 📌 Mémo important (alignement projet)
Pour éviter tout biais et garantir une vraie cohérence “retrieval → génération”, il est indispensable de se synchroniser avec le développeur du chatbot LLM afin de :

- **valider l’embedder cible** qui sera utilisé en production (retrieval, indexation, éventuellement reranking),
- s’assurer que **l’index** et les requêtes sont encodés avec **le même modèle** (sinon dégradation forte de la similarité),
- arbitrer entre :
  - performance sémantique,
  - coût CPU/GPU,
  - latence,
  - stabilité multilingue,
  - compatibilité avec le domaine juridique.

👉 Le choix final de l’embedder doit être **décidé au niveau architecture**, car il conditionne directement la qualité du RAG et les performances globales du système.
---

## 🔧 Notes notebook

- Si tu vois des erreurs `ModuleNotFoundError` (`corpus_loader`, `metrics`, `benchmark_queries`…), c’est juste que le notebook n’est pas lancé depuis la racine du projet : une cellule ci-dessous ajoute la racine au `sys.path`.
- Le fichier `juridical_dictionary.yml` doit être accessible depuis le répertoire courant (ou adapte le chemin dans la cellule dédiée).


## ⚙️ Setup (dépendances)

Si besoin, installe les dépendances utilisées dans ce notebook.


In [1]:
# Installation des dépendances nécessaires (à exécuter une seule fois par environnement)
%pip -q install sentence-transformers scikit-learn ipywidgets


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pathlib import Path
import sys, os

# On tente de se placer à la racine du projet pour rendre les imports locaux disponibles.
# Ajuste si ton notebook est dans un sous-dossier (ex: notebooks/ -> parent).
PROJECT_ROOT = Path.cwd()

# Exemple : si ce notebook est dans un dossier "notebooks", décommente :
# PROJECT_ROOT = Path.cwd().parent

sys.path.insert(0, str(PROJECT_ROOT))
print("PROJECT_ROOT =", PROJECT_ROOT)


PROJECT_ROOT = d:\-- Projet RAG Avocats --\codes_python\notebooks


## 📚 Dictionnaire métier

Le script charge `juridical_dictionary.yml`.  
Vérifie que ce fichier est présent dans le dossier courant (ou adapte le chemin ci-dessous).


## 🧪 Code (repris du script, identique hors adaptations notebook possibles)

In [3]:
# -*- coding: utf-8 -*-

"""
STAGE 9 – DENSE RETRIEVAL + QUERY UNDERSTANDING
"""

from corpus_loader import documents
from query_understanding import process_user_query, load_juridical_dictionary
from benchmark_queries import benchmark_queries
from metrics import recall_at_k, reciprocal_rank, ndcg_at_k
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity



# =========================================================
# 0. CHARGEMENT DU DICO DE juridical_dictionary.yml
# =========================================================
dictionary = load_juridical_dictionary("juridical_dictionary.yml")


# =========================================================
# 1. EMBEDDINGS
# =========================================================

model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

doc_texts = [doc["text"] for doc in documents]
doc_embeddings = model.encode(
    doc_texts,
    normalize_embeddings=True,
    show_progress_bar=True
)


def dense_search(query, k=10):
    query_embedding = model.encode(
        [query],
        normalize_embeddings=True
    )

    scores = cosine_similarity(query_embedding, doc_embeddings)[0]

    ranked = sorted(
        zip(documents, scores),
        key=lambda x: x[1],
        reverse=True
    )
    return ranked[:k]


# =========================================================
# 2. BENCHMARK
# =========================================================

def evaluate():
    recall_scores, mrr_scores, ndcg_scores = [], [], []

    for q in benchmark_queries:
        enriched = process_user_query(q["question"], dictionary)
        query = enriched["enriched_query"]

        results = dense_search(query, k=10)

        recall_scores.append(recall_at_k(results, q["relevant_keywords"], 10))
        mrr_scores.append(reciprocal_rank(results, q["relevant_keywords"]))
        ndcg_scores.append(ndcg_at_k(results, q["relevant_keywords"], 10))

    return {
        "Recall@10": sum(recall_scores) / len(recall_scores),
        "MRR": sum(mrr_scores) / len(mrr_scores),
        "nDCG@10": sum(ndcg_scores) / len(ndcg_scores),
    }


if __name__ == "__main__":
    print("\n=== Dense retrieval + requête enrichie ===")
    print(evaluate())

Corpus brut chargé : 4422 documents
Corpus filtré 'Code du travail' : 882 documents



Batches:   0%|          | 0/28 [00:00<?, ?it/s]


=== Dense retrieval + requête enrichie ===
{'Recall@10': 0.6666666666666666, 'MRR': 0.38095238095238093, 'nDCG@10': 0.4035162162237942}


## ✅ Conclusion (issue du script)

## Conclusion – Enseignements du benchmark RAG juridique

Ce benchmark avait pour objectif d’évaluer l’impact réel de différentes briques
(Retrieval lexical, filtrage métier, compréhension de requête, retrieval sémantique)
sur un cas d’usage juridique simple mais représentatif.

Le protocole expérimental est volontairement strict :
- même corpus,
- mêmes questions,
- mêmes métriques,
- seule la brique testée varie.

---

### Résumé des résultats

| Configuration                         | Recall@10 | MRR   | nDCG@10 |
|---------------------------------------|-----------|-------|---------|
| BM25 – requête brute                  | 0.33      | 0.33  | 0.33 |
| BM25 – requête enrichie métier        | 0.33      | 0.33  | 0.33 | script 5.
| BM25 filtré + requête enrichie        | 0.33      | 0.33  | 0.33 | script 6.
| Dense retrieval + requête enrichie    | 0.67      |0.38   | 0.40 | script 7.

---

### Lecture des résultats

1. **BM25 constitue une baseline solide en contexte juridique**  
   Les résultats montrent que le BM25 est déjà performant pour des requêtes
   fortement lexicalisées (articles, notions juridiques explicites).
   L’enrichissement de la requête n’améliore pas les métriques, ce qui est attendu :
   le moteur lexical disposait déjà des bons termes discriminants.

2. **Le filtrage métier seul n’est pas suffisant pour améliorer la pertinence**  
   Réduire le périmètre documentaire diminue le bruit, mais ne corrige pas
   le décalage entre langage utilisateur et langage juridique.
   Les faux négatifs critiques persistent.

3. **La compréhension métier devient décisive pour le retrieval sémantique**  
   L’introduction d’un dictionnaire juridique explicite permet de normaliser
   l’intention utilisateur et d’enrichir la requête avec des concepts juridiques pertinents.
   Cette structuration du langage est le facteur clé qui permet au retrieval dense
   d’exprimer son potentiel.

4. **Le gain observé sur le dense retrieval est significatif et mesurable**  
   Le rappel est doublé (0.33 → 0.67) et le ranking s’améliore.
   Ce gain ne provient pas du modèle seul, mais de l’association
   entre compréhension métier et similarité sémantique.

---

### Enseignement clé

> En RAG juridique, la performance ne vient pas d’un changement de modèle,
> mais de la structuration explicite du langage métier.
> Les algorithmes amplifient un signal correctement posé ;
> ils ne compensent pas une compréhension absente.

---

### Implications pour un système RAG en production

- Le BM25 reste un excellent garde-fou lexical.
- Le retrieval dense ne doit pas être utilisé seul.
- Une couche explicite de *query understanding* métier est indispensable.
- Les améliorations doivent être pilotées par des métriques mesurables,
  et non par intuition ou effet de mode.

Ce benchmark fournit ainsi une base saine, mesurable et améliorable
pour la construction d’un moteur RAG juridique fiable et contrôlé.